In [1]:
import pandas as pd
import psycopg2
import json
import datetime
%matplotlib inline

In [2]:
pd.set_option("display.max_colwidth", 500)

In [3]:
db_lexico = psycopg2.connect(**json.load(open('data/db_lexico.json')))

In [13]:
channels = pd.read_sql('select * from core_channel', db_lexico)

/usr/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [14]:
channels

,id,name,publicName,programId
0,1,cnews,CNEWS,516
1,2,bfmtv,BFMTV,515
2,3,franceinfo,franceinfo,262
3,4,lci,LCI,776
4,5,france2,France 2,257
5,6,france3parisidf,France 3 Paris Ile-de-France,273
6,7,c8,C8,513
7,8,tf1,TF1,1537
8,9,lcp,LCP,1545
9,10,france4,France 4,260


In [26]:

data = pd.read_sql(f'''select channel_id, lemma, count(*) from core_word 
                       where lemma='écologie' 
                       and "dateTime" >= '2022-04-01 00:00:00' 
                       and "dateTime" <= '2022-05-24 00:00:00'
                       group by channel_id, lemma
                       ''', db_lexico)

/usr/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 15 ms, sys: 7.77 ms, total: 22.8 ms
Wall time: 1min 25s


In [20]:
data_channel = data.merge(channels, left_on='channel_id', right_on='id')

In [22]:
data_channel

,channel_id,lemma,count,id,name,publicName,programId
0,7,écologie,66,7,c8,C8,513
1,6,écologie,237,6,france3parisidf,France 3 Paris Ile-de-France,273
2,10,écologie,60,10,france4,France 4,260
3,3,écologie,2001,3,franceinfo,franceinfo,262
4,4,écologie,618,4,lci,LCI,776
5,1,écologie,1166,1,cnews,CNEWS,516
6,9,écologie,1689,9,lcp,LCP,1545
7,5,écologie,383,5,france2,France 2,257
8,8,écologie,114,8,tf1,TF1,1537
9,2,écologie,826,2,bfmtv,BFMTV,515
